In [78]:
import numpy as np
import pandas as pd
import random as rand

import requests
import json
import re
from json import loads

In [104]:
bus_station = pd.read_csv('서울시버스정류소좌표데이터(2022.08.24).csv', encoding='euc-kr')
bus_station = bus_station.sample(n=100)
bus_station

,NODE_ID,ARS-ID,정류소명,좌표X,좌표Y
8862,120000420,21342,봉일시장,126.937442,37.485310
7297,116900129,17948,현대우성아파트,126.885495,37.511695
10600,123000278,24369,송파경찰서.오금역,127.126877,37.502789
10843,124000067,25167,고덕초등학교,127.165752,37.560086
6013,114000176,15279,서울남부지방법원.서울남부지방검찰청,126.864167,37.522419
...,...,...,...,...,...
1518,104000126,5219,노룬산시장앞,127.062953,37.536826
8872,120000423,21352,건영유치원,126.914997,37.485055
8570,119900235,20929,보라매공원입구,126.917648,37.496620
10310,122000707,23988,영동대교(가상),127.055431,37.526128


In [105]:
# Starting point는 0 부여
loc0_name = '강남역'
loc0_X = '127.028361548'
loc0_Y = '37.496486063'

In [106]:
# 목적지는 1부터 네이밍;; 목적지 개수는 (num-1) 개

num = 20 # 총 노드 개수

for i in range(1,num): # 1~(n-1)
    globals()[f'loc{i}_name'] = bus_station.iloc[i,2]
    globals()[f'loc{i}_X'] = bus_station.iloc[i,3]
    globals()[f'loc{i}_Y'] = bus_station.iloc[i,4]

In [107]:
print(loc1_name, loc1_X, loc1_Y)

봉일시장 126.9374424581 37.4853104251


In [112]:
url = "https://apis.openapi.sk.com/tmap/matrix?version=1"

payload = {
    "origins": [
        {
            "lon": str(globals()[f'loc{i}_X']),
            "lat": str(globals()[f'loc{i}_Y']),
        } for i in range(num)
    ],
    "destinations": [
        {
            "lon": str(globals()[f'loc{i}_X']),
            "lat": str(globals()[f'loc{i}_Y']),
        } for i in range(num)
    ],
    "transportMode": "car",
    "metric": "Recommendation"
}
headers = {
    "Accept": "application/json",
    "appKey": "l7xx4d23b06a733d4f2e9c5365ca49c3431a",
    "Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)

In [113]:
jsonObj = json.loads(response.text)
jsonObj

{'meta': {'status': 'Ok',
  'message': 'Ok',
  'moduleVersion': '1.21.0',
  'mapVersion': '20220830',
  'elapsedTime': 258,
  'realTimeTrafficDateTime': '2022-09-02 20:55',
  'patternTrafficDate': '2022-09-02',
  'customizationDateTime': '2022-09-02 20:55',
  'algorithm': 'CCH_EliminationTree',
  'metric': 'Recommendation',
  'requestId': '19787ea7-d75d-4f00-beee-72715281e023',
  'gasStationPriceDateTime': '',
  'transportMode': 'Car',
  'avoid': []},
 'origins': [{'link': {'linkId': '0', 'direction': 'TwoWay'},
   'poiId': '',
   'name': '',
   'coordinate': {'latitude': 37.496486063, 'longitude': 127.028361548},
   'rpFlag': ''},
  {'link': {'linkId': '0', 'direction': 'TwoWay'},
   'poiId': '',
   'name': '',
   'coordinate': {'latitude': 37.4853104251, 'longitude': 126.9374424581},
   'rpFlag': ''},
  {'link': {'linkId': '0', 'direction': 'TwoWay'},
   'poiId': '',
   'name': '',
   'coordinate': {'latitude': 37.5116946043, 'longitude': 126.8854946825},
   'rpFlag': ''},
  {'link':

In [114]:
jsonObj['matrixRoutes'][0]

{'status': 'Ok',
 'originIndex': 0,
 'destinationIndex': 0,
 'cost': 0,
 'duration': 0,
 'distance': 0.0,
 'toll': False}

In [115]:
duration_lst = []
distance_lst = []
for i in range(num**2):
    duration_lst.append(jsonObj['matrixRoutes'][i]['duration'])
    distance_lst.append(jsonObj['matrixRoutes'][i]['distance'])
duration = np.array(duration_lst).reshape(num,num)
distance = np.array(distance_lst).reshape(num,num)

In [119]:
print(duration.shape)
duration

(20, 20)


array([[   0, 2206, 2663, 1711, 2346, 2857,  736, 2415, 1894, 2623, 3477,
        2650,  317, 2068,  761, 1695,  689, 2836, 2602, 1176],
       [1599,    0, 1370, 1856, 3517, 1604, 1137, 3050, 2849, 3141, 1908,
        1237, 1484,  809, 1990, 1888, 1272,  877, 3121, 1026],
       [2397, 2355,    0, 2652, 3477,  567, 2080, 3432, 2620, 3305, 1193,
         352, 2540, 1143, 2201, 1582, 2204, 2709, 3285, 2013],
       [1517, 2907, 3281,    0, 1573, 3557, 1958, 1609, 2602, 1799, 3471,
        3343, 1297, 3020, 1803, 2697, 1876, 2776, 1762, 3157],
       [2328, 3097, 3319, 1329,    0, 3513, 2332, 1224, 2403, 1035, 4224,
        3158, 1923, 2822, 1730, 2494, 2475, 3465, 1377, 2500],
       [2689, 1412,  595, 2732, 3768,    0, 2371, 3651, 2880, 3525, 1233,
         329, 2475, 1434, 2493, 1905, 2495, 1509, 3504, 2304],
       [ 780, 1282, 2569, 1517, 2539, 2846,    0, 2711, 2476, 3069, 2759,
        2631,  664, 2077, 1171, 1975,  452, 1912, 3049, 1185],
       [2182, 2326, 2470, 1519, 1572, 288